In [1]:
from fastai.imports import *

np.set_printoptions(linewidth=130)

In [3]:
import os
from pathlib import Path

competition = "titanic"  # Change this to any Kaggle competition name
iskaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")

if iskaggle:
    path = Path(f"../input/{competition}")
else:
    import kaggle

    # Save data in `data/` directories, that are gitignored
    path = Path("data")

    if not path.exists():
        import zipfile

        kaggle.api.competition_download_cli(competition=competition, path=str(path))
        zipfile.ZipFile(f"{path}/{competition}.zip").extractall(path)

print(f"Ready for competition: {competition}")

Ready for competition: titanic


In [5]:
df = pd.read_csv(path / "train.csv")
tst_df = pd.read_csv(path / "test.csv")
modes = df.mode().iloc[0]

In [6]:
def proc_data(df):
    df["Fare"] = df.Fare.fillna(0)
    df.fillna(modes, inplace=True)
    df["LogFare"] = np.log1p(df["Fare"])
    df["Embarked"] = pd.Categorical(df.Embarked)
    df["Sex"] = pd.Categorical(df.Sex)


proc_data(df)
proc_data(tst_df)